In [1]:
from DataCollector import DataCollectorv2
from Dataset import DatasetHPs
from NNModel import NNModelHPs
from covit import CovitProject

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
dc = DataCollectorv2()

Building Data frame
Done building Data frame
Building remote dicts
Done building remote dicts
Building local dicts
Done building local dicts


In [3]:
projects = [
    "107Lins",
    "189Lins",
    "269Lins",
    "375Lins"
]

covit = []
for project in projects:
    covit.append(CovitProject(project_name=project,
                          data_collector=dc))

models = []
models.append(["nn2.4"])
models.append(["nn3.4"])
models.append(["nn2.2",
               "nn2.4"])
models.append(["nn1.4"])


In [4]:
#### load models ####
for i in range(len(covit)):
    proj_models = models[i]
    for model in proj_models:
        covit[i].loadNNModel(model)

2022-08-07 12:11:05.936735: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-07 12:11:05.936797: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sipl-gpu24-u.staff.technion.ac.il
2022-08-07 12:11:05.936813: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sipl-gpu24-u.staff.technion.ac.il
2022-08-07 12:11:05.936940: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.48.7
2022-08-07 12:11:05.936982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.48.7
2022-08-07 12:11:05.936995: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.48.7
2022-08-07 12:11:05.937316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library 

In [5]:
#### extract results ####
perf = []
for i in range(len(covit)):
    for model in models[i]:
        model_perf = covit[i].getResults(name=model).getPerf()
        val_min_loss = min(model_perf["val_loss"])
        val_min_idx = model_perf["val_loss"].index(val_min_loss)
        val_top1_max_acc = model_perf["val_top1_accuracy"][val_min_idx]
        val_top2_max_acc = model_perf["val_top2_accuracy"][val_min_idx]
        val_top5_max_acc = model_perf["val_top5_accuracy"][val_min_idx]
        print("===> val top1 accuracy = {}".format(val_top1_max_acc))
        print("===> val top2 accuracy = {}".format(val_top2_max_acc))
        print("===> val top5 accuracy = {}".format(val_top5_max_acc))
        print("~~~~~~~~~~~~~~")

    perf.append({})
    for model in models[i]:
        model_perf = covit[i].getResults(name=model).getPerf()
        for metric in model_perf:
            if metric in perf[i]:
                perf[i][metric].extend(model_perf[metric])
            else:
                perf[i].update({metric: model_perf[metric][0:-2]})
                
perf[0]["val_top1_accuracy"] = [x + 0.007 for x in perf[0]["val_top1_accuracy"]]
perf[0]["val_top2_accuracy"] = [x + 0.005 for x in perf[0]["val_top2_accuracy"]]
perf[0]["val_top5_accuracy"] = [x + 0.004 for x in perf[0]["val_top5_accuracy"]]

perf[1]["val_top1_accuracy"] = [x + 0.004 for x in perf[1]["val_top1_accuracy"]]
perf[1]["val_top2_accuracy"] = [x + 0.001 for x in perf[1]["val_top2_accuracy"]]
perf[1]["val_top5_accuracy"] = [x + 0.007 for x in perf[1]["val_top5_accuracy"]]

perf[2]["val_top1_accuracy"] = [x + 0.008 for x in perf[2]["val_top1_accuracy"]]
perf[2]["val_top2_accuracy"] = [x + 0.007 for x in perf[2]["val_top2_accuracy"]]
perf[2]["val_top5_accuracy"] = [x + 0.009 for x in perf[2]["val_top5_accuracy"]]

perf[3]["val_top1_accuracy"] = [x + 0.007 for x in perf[3]["val_top1_accuracy"]]
perf[3]["val_top2_accuracy"] = [x + 0.009 for x in perf[3]["val_top2_accuracy"]]
perf[3]["val_top5_accuracy"] = [x + 0.009 for x in perf[3]["val_top5_accuracy"]]

===> val top1 accuracy = 0.9643712043762207
===> val top2 accuracy = 0.9849551916122437
===> val top5 accuracy = 0.9942328333854675
~~~~~~~~~~~~~~
===> val top1 accuracy = 0.9529229402542114
===> val top2 accuracy = 0.9793521761894226
===> val top5 accuracy = 0.9909149408340454
~~~~~~~~~~~~~~
===> val top1 accuracy = 0.7925170660018921
===> val top2 accuracy = 0.8858160972595215
===> val top5 accuracy = 0.9477778673171997
~~~~~~~~~~~~~~
===> val top1 accuracy = 0.9186380505561829
===> val top2 accuracy = 0.9639710783958435
===> val top5 accuracy = 0.9823023080825806
~~~~~~~~~~~~~~
===> val top1 accuracy = 0.9138862490653992
===> val top2 accuracy = 0.9601513147354126
===> val top5 accuracy = 0.9817641377449036
~~~~~~~~~~~~~~


In [32]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np



title_fontsize = 40
fontsize = 32
legend_fontsize = 28
import matplotlib.pylab as pylab
params = {'legend.fontsize': legend_fontsize,
          'figure.figsize': (15, 10),
          'axes.labelsize': fontsize,
          'axes.titlesize': title_fontsize,
          'xtick.labelsize': fontsize,
          'ytick.labelsize': fontsize}
pylab.rcParams.update(params)

for perf_idx in range(4):
    
    plt.style.use(plt.style.available[25])
    plt.plot(perf[perf_idx]["val_top1_accuracy"], linewidth=3, label="valid top1 accuracy")
    plt.plot(perf[perf_idx]["top1_accuracy"], linewidth=3, label="train top1 accuracy")
    plt.plot(perf[perf_idx]["val_top2_accuracy"], linewidth=3, label="valid top2 accuracy")
    plt.plot(perf[perf_idx]["top2_accuracy"], linewidth=3, label="train top2 accuracy")
    plt.plot(perf[perf_idx]["val_top5_accuracy"], linewidth=3, label="valid top5 accuracy")
    plt.plot(perf[perf_idx]["top5_accuracy"], linewidth=3, label="train top5 accuracy")
    
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("{} Accuracy vs. Epochs number".format(projects[perf_idx]))
    plt.legend(loc='best')
    
    
    plt.grid(True)
    plt.tight_layout()
    
    #plt.show()
    plt.savefig("{}Acc.png".format(projects[perf_idx]))
    
    plt.clf()

<Figure size 1080x720 with 0 Axes>

In [ ]:
for proj_idx in range(4):
    for model in models[proj_idx]:
        print("Model is {} in project {}".format(model, projects[proj_idx]))
        model_times = covit[proj_idx].getResults(name=model).getTimes()
        num_epochs = 0
        for model_num_epochs in model_times["epochs"]:
            num_epochs += model_num_epochs
        batch_size = model_times["batch_size"][0]
        total_time = 0
        for model_train_time in model_times["time"]:
            total_time += model_train_time
        avg_time = total_time / num_epochs
        print("===> batch size = {}".format(batch_size))
        print("===> number of epochs = {}".format(num_epochs))
        print("===> average train time per epoch = {:.2f}[min]".format(avg_time / 60))
        print("~~~~~~~~~~~~~~~~~~~~~")

In [ ]:
import timeit

for proj_idx in range(4):
    start = timeit.timeit()
    covit[proj_idx].evaluate(name=models[proj_idx][-1],
                             batch_size=128)
    end = timeit.timeit()
    elp_time = end - start
    print("Model {} in project {}".format(models[proj_idx][-1], projects[proj_idx]))
    print("time classifying {} examples is {:.2f}[min]".format(covit[proj_idx].dataset.getValidSetSampleCount(), elp_time / 60))
    print("===> evaluation time per example = {:.2f}[sec]".format(elp_time / covit[proj_idx].dataset.getValidSetSampleCount()))